In [ ]:
!pip install roboflow
!pip install ultralytics
!pip install torch torchvision pyyaml

import os
from google.colab import drive
import yaml
import torch
from ultralytics import YOLO

  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached ultralytics_thop-2.0.14-py3-none-any.whl.metadata (9.4 kB)
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/922.2 kB ? eta -:--:--
   - -------------------------------------- 30.7/922.2 kB ? eta -:--:--
   - ------------------------------------- 41.0/922.2 kB 991.0 kB/s eta 0:00:01
   - ------------------------------------- 41.0/922.2 kB 991.0 kB/s eta 0:00:01
   - ------------------------------------- 41.0/922.2 kB 991.0 kB/s eta 0:00:01
   --- ----------------------------------- 71.7/922.2 kB 393.8 kB/s eta 0:00:03
   ---- --------------------------------- 102.4/922.2 kB 492.8 kB/s eta 0:00:02
   ------- ------------------------------ 174.1/922.2 kB 655.4 kB/s eta 0:00:02
   ------- ------------------------------ 174.1/922.2 kB 655.4 kB/s eta 0:0

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="YOUR_API_KEY")
project = rf.workspace("400mhdataset").project("400mh-dataset-model")
version = project.version(1)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to 400mH-Dataset-Model-1 in yolov11:: 100%|██████████| 821/821 [00:00<00:00, 1305.34it/s]


In [ ]:
DATA_PATH = '/content/400mH-Dataset-Model-1/data.yaml'
WEIGHTS_PATH = 'yolo11m.pt'
EPOCHS=100
BATCH_SIZE=16
IMG_SIZE=640

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

In [ ]:
if not os.path.exists(WEIGHTS_PATH):
  print(f'Downloading {WEIGHTS_PATH}...')
  !wget https://github.com/ultralytics/assets/releases/download/v11.0/{WEIGHTS_PATH} -O {WEIGHTS_PATH}
else:
  print(f'{WEIGHTS_PATH} already exists')

In [ ]:
def prepare_data_yaml(data_path):
    """
    Ensure data.yaml is correctly formatted for YOLOv11
    """
    if not os.path.exists(data_path):
        data_config = {
            'train': '/content/400mH-Dataset-Model-1/train',
            'val': '/content/400mH-Dataset-Model-1/valid',
            'nc': 2,
            'names': ['athlete', 'hurdle']
        }
        with open(data_path, 'w') as f:
            yaml.dump(data_config, f)
        print("Created new data.yaml")
    else:
        with open(data_path, 'r') as f:
            data_config = yaml.safe_load(f)

        # Verify required fields
        assert 'train' in data_config and 'val' in data_config, "Missing train/val paths"
        assert data_config['nc'] == 2, "Number of classes should be 2"
        assert data_config['names'] == ['athlete', 'hurdle'], "Class names mismatch"
        print("Verified existing data.yaml")

    return data_config

# Prepare and check data
data_config = prepare_data_yaml(DATA_PATH)

In [ ]:
model = YOLO(WEIGHTS_PATH)

# Train the model
results = model.train(
    data=DATA_PATH,
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    device=device,
    project='/content/400mH-Dataset-Model-1/yolov11_training/runs',  # Save to Drive
    name='exp',
    exist_ok=True,
    plots=True,
    save=True,
    patience=10,
    optimizer='AdamW',
    lr0=0.001,
)

# Print results
print(f"Training completed! Best mAP: {results.metrics['mAP50-95']:.4f}")